In [4]:
import numpy as np


def entropy(var):
    N = var.shape[0]
    values, counts = np.unique(var, return_counts=True )
    
    ent = 0.0
    
    for i in counts:
        p = i/N
        ent += (p * np.log2(p))
        
    return -ent


Y = np.array([1,0,0,1,0,1,0,1])

X = np.array([1,1,1,1,1,1,1])


entropy(Y)

1.0

In [5]:
entropy(X)


-0.0

CODE: Split Data

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('file.csv')
output_feature='buys_computer'

In [8]:
df.head()

,age,income,student,credit_rating,buys_computer
0,<=30,high,no,fair,no
1,<=30,high,no,excellent,no
2,31...40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>40,low,yes,fair,yes


In [9]:
def divide_data(data, feature):
    # here we are working data frames.
    
    DATA = {}
    
    feat_values = list(data[feature].value_counts().index)
    occurence = list(data[feature].value_counts())
    
    for val in feat_values:
        DATA[val] = {'data' : pd.DataFrame([], columns = data.columns), 'len': 0}
        
    
    for ix in range(data.shape[0]):
        val = data[feature].iloc[ix]
        
        DATA[val]['data'] = DATA[val]['data']._append(data.iloc[ix])
    
        idx = feat_values.index(val)
        DATA[val]['len']  = occurence[idx]
    
    return DATA

In [10]:
divide_data(df,'income')


{'medium': {'data':         age  income student credit_rating buys_computer
  3       >40  medium      no          fair           yes
  7      <=30  medium      no          fair            no
  9       >40  medium     yes          fair           yes
  10     <=30  medium     yes     excellent           yes
  11  31...40  medium      no     excellent           yes
  13      >40  medium      no     excellent            no,
  'len': 6},
 'high': {'data':         age income student credit_rating buys_computer
  0      <=30   high      no         fair             no
  1      <=30   high      no     excellent            no
  2   31...40   high      no          fair           yes
  12  31...40   high     yes          fair           yes,
  'len': 4},
 'low': {'data':        age income student credit_rating buys_computer
  4      >40    low     yes          fair           yes
  5      >40    low     yes     excellent            no
  6  31...40    low     yes     excellent           yes
  8     

CODE: Information Gain


In [11]:
def information_gain(data, feature):
    examples = data.shape[0]
    
    DATA = divide_data(data, feature)
    
    keys = DATA.keys()
    
    
    ent_of_children = 0.0
    
    for key in keys:
        ent_of_children +=  ( (DATA[key]['len']/examples) * entropy(DATA[key]['data'][output_feature]) )
        
    info_gain = entropy(data[output_feature]) - ent_of_children
    return info_gain

In [12]:
information_gain(df, 'income')

0.02922256565895487

In [13]:
information_gain(df, 'age')

0.24674981977443933

In [14]:
information_gain(df, 'credit_rating')

0.2158781408080388

In [15]:
information_gain(df, 'student')

0.15183550136234159

Constructing Decision Tree

In [16]:
class DecisionTree:
    
    # constructor
    def __init__(self, depth=0, max_depth=5):
        # Creating a Node
        self.children = {}
        self.fkey = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
        
    def train(self, data):
        features = data.columns[:-1]
        
        info_gains = []
            
        for f in features:
            i_gain = information_gain(data, f)
            info_gains.append(i_gain)
            
        # finding the best feature
        self.fkey = features[np.argmax(info_gains)]
        
        #Spliting the Data
        DATA = divide_data(data, self.fkey)
        
        
        
        # Giving a target label to the Node
        labels = list(data[output_feature].value_counts().index)
        freq = list(data[output_feature].value_counts().values)
        
        self.target = labels[np.argmax(freq)]
        

        ###### STOPPING CONDITIONS ######
        
        have_data = 0
        keys = DATA.keys()
        
        for key in keys:
            if DATA[key]['len'] > 0:
                have_data +=1
        
        # 1. If it is pure node 
        if have_data<2:
            print("\t"*self.depth + "Leaf Node : "+ self.target)
            return 
        
    
        # 2. Early Stop if you have reached max depth
        if(self.depth >= self.max_depth):
            print("\t"*self.depth + "Leaf Node : "+ self.target)
            return
        
        
        print("\t"*self.depth + "Node : ", self.fkey)
        
        
        # Recursively train child Node
        for key in keys:
            new_data = DATA[key]['data']
            self.children[key] = DecisionTree(depth = self.depth + 1)
            self.children[key].train(new_data)
            
        return  
    
    
    def predict(self, test):
        if self.children == {}:
            return self.target
        print("\t"*self.depth + "Testing Node for - ", self.fkey);
        return self.children[test[self.fkey][0]].predict(test)


Model


In [17]:
model = DecisionTree()

In [18]:
model.train(df)

Node :  age
	Node :  student
		Leaf Node : no
		Leaf Node : yes
	Node :  credit_rating
		Leaf Node : yes
		Leaf Node : no
	Leaf Node : yes


In [19]:
model

In [20]:
model.target

'yes'

In [21]:
model.fkey

'age'

In [22]:
model.children

{'<=30': <__main__.DecisionTree at 0x21e8e224e10>,
 '>40': <__main__.DecisionTree at 0x21e8f02dc10>,
 '31...40': <__main__.DecisionTree at 0x21e87e19050>}

In [23]:
model.children['>40']

In [24]:
model.children['>40'].fkey

'credit_rating'

In [25]:
model.children['>40'].children

{'fair': <__main__.DecisionTree at 0x21e8f03add0>,
 'excellent': <__main__.DecisionTree at 0x21e8f046390>}

In [26]:
model.children['>40'].children['fair'].children

{}

In [27]:
model.children['31...40'].target

'yes'

In [28]:
model.children['31...40'].children

{}

Prediction

In [29]:
test_data=[]
x_test = pd.DataFrame([['<=30', 'low', 'no', 'fair']], columns=list(df.columns.values[:-1]))

In [30]:
x_test

,age,income,student,credit_rating
0,<=30,low,no,fair


In [31]:
model.predict(x_test)

Testing Node for -  age
	Testing Node for -  student


'no'